# Visualización de datos a través de estratos

Trabajaremos con el conjunto de datos de **Google Play Store Apps.**

In [146]:
import altair as alt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [147]:
data = pd.read_csv("https://raw.githubusercontent.com/lihkir/Uninorte/main/AppliedStatisticMS/DataVisualizationRPython/Lectures/Python/PythonDataSets/googleplaystore.csv")
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


## Preprocesamiento 

Antes de proceder con las visualizaciones, se debe realizar un poco de preprocesamiento en los datos. Se identificaran si existen datos perdidos o NA en el conjunto de datos.

In [148]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [149]:
# detección de valores NA
data.isna().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

En el conjunto de datos existen 5 caracteristicas con valores perdidos. La mas destacable es **Rating** la cual tiene 1474 datos perdidos. Por efectos del presente análisis se eliminarán todos los registros con datos nulos.

In [150]:
# eliminar datos perdidos
data = data.dropna()

In [151]:
data.isna().sum()

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              0
Price             0
Content Rating    0
Genres            0
Last Updated      0
Current Ver       0
Android Ver       0
dtype: int64

Al verficar nuevamente la cantidad de datos perdidos se puede ver que estos han sido eliminados de manera exitosa del conjunto de datos.

## Visualizaciones

Ahora se procede a realizar las visualizaciones para el conjunto de datos.

* Gráfico de barras de un número de aplicaciones estratificado por cada categoría Content Rating (calificado por Everyone/Teen).

In [152]:
data["Content Rating"].unique()

array(['Everyone', 'Teen', 'Everyone 10+', 'Mature 17+',
       'Adults only 18+', 'Unrated'], dtype=object)

La variables **Content Rating** tiene 5 categorías, las cuales indica el público objetivo de cada aplicación. A continuación se realizará un grafico de barras para visualizar la cantidad de aplicaciones que hay por Clasificación de contenido.

In [153]:
alt.Chart(data).mark_bar().encode(
    y='count()',
    x='Content Rating:N',
    tooltip=['Content Rating:N', 'count()']
).properties(
    width=400,
    title='Número de aplicaciones por Content Rating'
)

alt.Chart(...)

In [154]:
data["Content Rating"].value_counts()

Content Rating
Everyone           7414
Teen               1084
Mature 17+          461
Everyone 10+        397
Adults only 18+       3
Unrated               1
Name: count, dtype: int64

Se logra observar que la mayoría de las aplicaciones están dirigidas a todo público (Everyone), seguido muy lejos por aplicaciones dirigidas hacia Adolecentes (Teen). En el grafico aparentement no hay registros de aplicaciones para mayores de edad (Adults only 18+) o sin clasificación (Unrated), pero al detallar el conteo por categoría si se logran identificar 3 registros para las aplicaciones de mayores de edad, y 1 registros para aplicaciones sin clasificación.

* Mapa de calor del número de aplicaciones estratificadas por app Category y rangos de rangos segmentados por Rating

In [155]:
alt.Chart(data).mark_rect().encode(
    alt.X('Rating:O'),
    alt.Y('Category:N'),
    alt.Color('count()', scale=alt.Scale(scheme='greenblue'), legend=alt.Legend(title='Total Apps')),
    tooltip=['Category:N', 'Rating:O', 'count()']
).properties(title='Número de aplicaciones por Category y Rating')

alt.Chart(...)

En el mapa de calor se logra apreciar que para las categorias **FAMILY**,  **GAME** y **TOOLS** existe una gran cantidad de aplicaciones con un Rating entre 4 y 4.7. Existen muchas zonas blancas por debajo de un Rating de 3.7, lo que indica que no muchas aplicaciones han sido calificadas con puntajes bajos, y por lo general tienen calificaciones regulares o altas.

* Gráficos vinculados dinámicamente

In [156]:
# Configurar selección
selected_region = alt.selection(type="single", encodings=['x'])

heatmap = alt.Chart(data).mark_rect().encode(
    alt.X('Rating:O'),
    alt.Y('Category:N'),
    alt.Color('count()', scale=alt.Scale(scheme='greenblue'), legend=alt.Legend(title='Total Apps')),
    tooltip=['Category:N', 'Rating:O', 'count()']
)

In [157]:
circles = heatmap.mark_point().encode(
    alt.ColorValue('grey'),
    alt.Size('count()', legend = alt.Legend(title='Records in Selection'))
).transform_filter(
    selected_region
)

In [158]:
bars = alt.Chart(data).mark_bar().encode(
    x = 'Content Rating:N',
    y = 'count()',
    color = alt.condition(selected_region, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).properties(
    width=350
).add_selection(selected_region)

heatmap + circles | bars

alt.HConcatChart(...)

En el gráfico anterior, se logra visualizar como al escoger una categoria de Contente Rating, se filtran en el mapa de calor la catidad de aplicaciones en esa selección que caen en una Categoria y con un Rating especifico. 
Por ejemplo, al seleccionar la clasificación **Teen**, se observa que la mayoria de aplicaciones en esta clasificación son de la Categoria **GAME**, seguido de la categoría **FAMILY**, con puntuaciones muy altas. Otro ejemplo es la clasificación mayores de 17 (Mature +7), en la que se observa que hay una gran cantidad de aplicaciones pertenecen a la categoria **DATING**, con puntuaciones entre 4 y 4.2.  